In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim

# Завантаження даних
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
dataset = pd.read_csv(url, header=None)

# Відокремлення ознак і цільової змінної
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

# Побудувати кореляційну матрицю
correlation_matrix = X.corr()
print("Correlation Matrix:")
print(correlation_matrix)

# Вибір ознак для тренування моделі
selected_features = correlation_matrix.abs().nlargest(7, correlation_matrix.columns[-1]).index
X_selected = dataset[selected_features]
X_train_selected, X_val_selected, y_train_selected, y_val_selected = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Нормалізація вибраних даних
scaler_selected = MinMaxScaler()
X_train_selected_normalized = scaler_selected.fit_transform(X_train_selected)
X_val_selected_normalized = scaler_selected.transform(X_val_selected)

# Конвертуємо дані у тензори PyTorch
X_train_selected_tensor = torch.tensor(X_train_selected_normalized, dtype=torch.float32)
y_train_selected_tensor = torch.tensor(y_train_selected.values, dtype=torch.float32).unsqueeze(1)
X_val_selected_tensor = torch.tensor(X_val_selected_normalized, dtype=torch.float32)
y_val_selected_tensor = torch.tensor(y_val_selected.values, dtype=torch.float32).unsqueeze(1)

# Визначення архітектури нейронної мережі
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Ініціалізація моделі та втрат
input_size_selected = X_train_selected.shape[1]
model_selected = NeuralNetwork(input_size_selected)
criterion_selected = nn.BCELoss()
optimizer_selected = optim.Adam(model_selected.parameters(), lr=0.001)

# Тренування моделі
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer_selected.zero_grad()
    outputs_selected = model_selected(X_train_selected_tensor)
    loss_selected = criterion_selected(outputs_selected, y_train_selected_tensor)
    loss_selected.backward()
    optimizer_selected.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss_selected.item():.4f}')

# Оцінка моделі
with torch.no_grad():
    outputs_selected = model_selected(X_val_selected_tensor)
    predicted_selected = torch.round(outputs_selected)
    accuracy_selected = (predicted_selected == y_val_selected_tensor).sum().item() / y_val_selected_tensor.size(0)
    print(f'Accuracy on validation set with selected features: {accuracy_selected:.2f}')


Correlation Matrix:
          0         1         2         3         4         5         6  \
0  1.000000  0.129459  0.141282 -0.081672 -0.073535  0.017683 -0.033523   
1  0.129459  1.000000  0.152590  0.057328  0.331357  0.221071  0.137337   
2  0.141282  0.152590  1.000000  0.207371  0.088933  0.281805  0.041265   
3 -0.081672  0.057328  0.207371  1.000000  0.436783  0.392573  0.183928   
4 -0.073535  0.331357  0.088933  0.436783  1.000000  0.197859  0.185071   
5  0.017683  0.221071  0.281805  0.392573  0.197859  1.000000  0.140647   
6 -0.033523  0.137337  0.041265  0.183928  0.185071  0.140647  1.000000   
7  0.544341  0.263514  0.239528 -0.113970 -0.042163  0.036242  0.033561   

          7  
0  0.544341  
1  0.263514  
2  0.239528  
3 -0.113970  
4 -0.042163  
5  0.036242  
6  0.033561  
7  1.000000  
Epoch [100/1000], Loss: 0.6442
Epoch [200/1000], Loss: 0.6222
Epoch [300/1000], Loss: 0.5348
Epoch [400/1000], Loss: 0.4787
Epoch [500/1000], Loss: 0.4602
Epoch [600/1000], Loss: